# Scalable K-means++ Re-implementation

### Abstract

In *k-means* clustering, we are given a set of $n$ data points in d-dimensional space $\mathbb{R}^d$ and an integer $k$ and the problem is to determine a set of $k$ points in $\mathbb{R}^d$, called centers, so as to minimize the mean squared distance from each data point to its nearest center. One of the major issues of *k-means* algorithm is to find a good initialization for obtaining a good final solution. The paper, "Scalable k-means++", introduces a new type of initialization *k-means||* as an improved version of an already existed initialization method *k-means++*, by taking advantage of parallel computing to resolve the issue of scalability in *k-means++* initialization due to its inherent sequential nature. In this final project, our group has managed to re-implement this paper using the techniques we have learnt in STA 663.

**Key words**: *k-means, clustering, k-means++, k-means||, parallel computing* 

## 1. K-means Algorithm

### 1.1. Notation and background

Let $X=\{x_1,...,x_n \}$ be a set of points in the d-dimensional Euclidean space and let k be apositive integer specifying the number of clusters. Let $||x_i-x_j||$ denote the Eculidean distance between $x_i$ and $x_j$. For a point $x$ and a subset $Y \subseteq X$ of points, the distance is defined as $d(x,Y) = \text{min}_{y \in Y} ||x-y||$. For a subset $Y \subseteq X$ of points, lets is *centroid* be given by

$$\text{centroid}(Y) = \frac{1}{|Y|} \sum_{y \in Y}y$$

Let $\mathcal{C} = \{c_1, ..., c_k \}$ be a set of points and let $Y \subseteq X$. We define the *cost* of Y with respect to $\mathcal{C}$ as 

$$\phi_Y(C)=\sum_{y\in Y} d^2(y,C) = \sum_{y \in Y} \min_{i=1,...,k}||y-c_i||^2$$

The goal of *k-means* clustering is to choose a set $\mathcal{C}$ of $k$ centers to minimize $\phi_X(C)$.

### 1.2. Algorithms

#### 1.2.1. K-means++

The main idea in *k-means++* initialization algorithm is to choose the centers one by one in a controlled fashion, where the current set of chosen centers will stochastically *bias* the choice of the next center. The central drawback of *k-means++* initialization from a scalability point of view is its inherent *sequential* nature: the choice of the next center depends on the current set of centers.

#### 1.2.2. K-means++ Algorithm

> 1: $\mathcal{C} \leftarrow$ sample a point uniformly at random from $X$

> 2: **while** |$\mathcal{C}$| < $k$ **do**

>> 3: Sample $x \in X$ with probability $\frac{d^2(x,C)}{\phi_X(C)}$

>> 4: $\mathcal{C} \leftarrow C \bigcup \{x\}$

> 5: **end while**


#### 1.2.3. K-means||

The paper proposes an improved initialization method, called *k-means||*, which uses parallel computing to overcome the inherent *sequential* nature of *k-means++*. *k-means||* uses an *oversampling factor* $l=\Omega(k)$, such that the algorithm picks an inital center (uniformly at random) and computes $\psi$, the initial cost of the clustering after this selection. It then proceeds in $\log \psi$ iterations, where in each iteration, given the current set $\mathcal{C}$ of centers, it samples each x with probability $\frac{ld^2(x,C)}{\phi_X(C)}$. The sampled points are then added to $\mathcal{C}$, the quantity of $\phi_X(C)$ updated, and the iteration continued.

#### 1.2.4. K-means|| Algorithm

> 1: $\mathcal{C} \leftarrow$ sample a point uniformly at random from $X$

> 2: $\psi \leftarrow \Phi_X(C)$

> 3: **for** $O(\log\psi)$ **do**

>> 4: $\mathcal{C}^\prime \leftarrow$ sample each point $x\in X$ independently with probability $p_x=\frac{ld^2(x,C)}{\phi_X(C)}$

>> 5: $\mathcal{C} \leftarrow \mathcal{C} \bigcup \mathcal{C}^\prime$

> 6: **end for**

> 7: For $x \in \mathcal{C}$, set $w_x$ to be the number of points in $X$ closer to $x$ than any other point in $\mathcal{C}$

> 8: Recluster the weighted points in $\mathcal{C}$ into $k$ clusters.


## 2. A Parallel Implementation

The paper also discusses a parallel implementation of *k-means||* in the MapReduce model of computation. Referring to the *k-means||* algorithm, we observe that Step 4 is very simple in MapReduce: each mapper can sample *independently* and Step 7 is equally simple given a set $\mathcal{C}$ of centers. Given a small set $\mathcal{C}$ of centers, computing $\phi_X(C)$ is also easy: each mapper working on an input partition $X^\prime \subseteq X$ can compute $\phi_{X^\prime}(C)$ and the reducer can simply add these values from all mapper to obtain $\phi_X(C)$. This takes care of Step 2 and the update to $\phi_X(C)$ neded for the iteration in Steps 3 to 6.

## 3. Experimental Setup

It is noted that this paper was published in 2012, and we are re-implementing it in 2018. According to the paper, they used a single workstation with quad-core 2.5GHz processors and 16Gb of memory. The parallel algorithms were run using a Hadoop cluster of 1968 nodes, each with two quad-core 2.5GHz processors and 16GB of memory. On the other hand, we are using Duke Virtual Machine to carry out the experiments. The difference in specs may result in different performance.

### 3.1. Datasets

We are using the same datasets used by the original paper; namely, a synthetic dataset *GAUSSMIXTURE*, and two real-life datasets, *SPAM* and *KDDCup1999*, from UC Irvine Machine Learning repository (archive.ics.uci.edu/ml/datasets.html).

To generate the synthetic *GAUSSMIXTUE* dataset, we sampled $k$ centers from a 15-dimensional spherical Gaussian distribution with mean at the origin and variance $R \in \{1,10,100\}$. We then added points from Gaussian distribution of unit ariance around each center. Given the k centers, this is a mixture of $k$ spherical Gaussians with equal weights. Note that the Gaussians are separated in terms of probability mass, and thus the value of the optimal k-clustering can be approximated using the centers of these Gausians. The number of smapled points from this mixture of Gaussians is $n=10,000$.

On the other hand, the real-life dataset *SPAM* consists of 4601 points in 58 dimensions and represents features available to an e-mail spam detection system. The *KDDCup1999* dataset consists of 4.8M pints in 42 dimensions and was used for the 1999 KDD Cup.

## 4. Re-implementation